<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e622c16b5df3c6ff71c2e72fb96e590925a107a3e190ef54ea833f593dd6b95c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-18 11:09:07
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.39 C
-------------------
Today PnL: -38.55 K (-0.28%)
Current PnL: -28.15 L (-18.18%)
CY Booked + Current PnL: -13.86 L (-8.95%)
-------------------
Total profit:  1.31 L
Total loss:  -29.46 L
-------------------
Total Booked + Current PnL: 13.12 L (10.27%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.29%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.78 L (66.8%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,1.51,9.63,9.18,19.69,19297.0,18468.0,210210.0,-1.08,59.0,X-MC,2.52,78.0,0.96,1.52,17.08,XY25,NTT,AC
77,TTKPRESTIG,770.00,-0.89,-19.89,24.94,0.09,20135.0,-20042.0,80735.0,87.41,28.0,M-SC,2.39,253.0,-1.00,0.58,6.07,OX40N,NTT,DURABLES
50,MASFIN,398.61,-0.60,-4.12,27.29,22.05,25638.0,-4035.0,93945.0,-17.34,49.0,H-SC,6.78,164.0,-0.16,0.68,36.56,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.39,-16.07,19.17,0.02,28085.0,-28053.0,146506.0,123.21,42.0,M-SC,13.67,234.0,-1.00,1.06,27.43,OX40N,NTT,PAINTS
43,ITC,452.00,0.35,-1.67,12.66,10.78,29916.0,-4011.0,236307.0,-41.56,45.0,X-LC,2.05,5.0,-0.13,1.71,3.48,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,2227.21,7.18,-44.64,205.37,69.05,151284.0,-59402.0,73664.0,-55.25,59.0,H-SC,22.33,140.0,-0.39,0.53,13.61,SR,ATH,IT
6,ASIANTILES,137.00,3.60,-11.54,105.37,81.67,87443.0,-10823.0,82987.0,7461.11,60.0,L-SC,12.43,271.0,-0.12,0.60,60.71,XR,NTT,CERAMICS
82,VOLTAS,1530.00,1.51,9.63,9.18,19.69,19297.0,18468.0,210210.0,-1.08,59.0,X-MC,2.52,78.0,0.96,1.52,17.08,XY25,NTT,AC
71,TANLA,1943.92,1.21,-36.92,248.25,119.69,424033.0,-99952.0,170809.0,-36.34,44.0,H-SC,7.84,156.0,-0.24,1.24,34.94,AR,ATH,IT
41,INFY,1972.00,1.19,10.36,21.65,34.24,74756.0,32406.0,345294.0,-14.21,65.0,X-LC,4.56,2.0,0.43,2.50,19.50,X40,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-3.44,-63.50,174.16,0.07,87200.0,-87108.0,50069.0,1734.60,50.0,L-SC,2.35,268.0,-1.00,0.36,24.52,OX40N,NTT,JEWELLERY
78,UNITDSPR,1644.0,-2.80,1.74,18.70,20.77,44547.0,4076.0,238220.0,-7.16,39.0,X-MC,2.32,62.0,0.09,1.72,8.78,X40N,NTT,BREWERIES
80,VALIANTORG,838.0,-2.55,-46.07,222.93,74.15,196691.0,-75375.0,88230.0,-258.45,40.0,H-SC,22.86,149.0,-0.38,0.64,13.10,XR,NTT,CHEMICALS
14,CAMPUS,393.0,-2.31,-13.32,53.76,33.28,78186.0,-22339.0,145436.0,-30.19,32.0,M-SC,2.28,222.0,-0.29,1.05,12.72,XY24,NTT,FOOTWEAR
16,CERA,9475.0,-2.04,-30.44,78.10,23.89,95563.0,-53543.0,122360.0,-32.94,40.0,H-SC,11.04,157.0,-0.56,0.89,5.67,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.79,-2.18,112.55,107.91,162545.0,-3224.0,144420.0,-20.55,56.0,M-SC,12.55,216.0,-0.02,1.04,7.68,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.39,-16.07,19.17,0.02,28085.0,-28053.0,146506.0,123.21,42.0,M-SC,13.67,234.0,-1.00,1.06,27.43,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.89,-19.89,24.94,0.09,20135.0,-20042.0,80735.0,87.41,28.0,M-SC,2.39,253.0,-1.00,0.58,6.07,OX40N,NTT,DURABLES
66,SIS,528.00,0.11,-24.13,59.83,21.26,50400.0,-26793.0,84239.0,1987.56,50.0,H-SC,4.96,166.0,-0.53,0.61,13.82,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.29,-26.10,53.55,13.47,106718.0,-70380.0,199287.0,-69.67,32.0,H-SC,6.05,158.0,-0.66,1.44,1.33,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.92,8.1,59.36,72.27,108728.0,13719.0,183167.0,-7.44,69.0,M-LC,2.8,99.0,0.13,1.33,14.77,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.92,8.10,59.36,72.27,108728.0,13719.0,183167.0,-7.44,69.0,M-LC,2.80,99.0,0.13,1.33,14.77,XR,NTT,IT
25,FINCABLES,1641.55,0.79,-2.18,112.55,107.91,162545.0,-3224.0,144420.0,-20.55,56.0,M-SC,12.55,216.0,-0.02,1.04,7.68,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.02,-3.85,115.01,106.72,136381.0,-4754.0,118582.0,-52.34,37.0,H-SC,3.44,139.0,-0.03,0.86,18.62,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.32,-3.49,40.53,35.63,81976.0,-7315.0,202259.0,-17.18,37.0,H-MC,2.77,119.0,-0.09,1.46,12.60,AR,ATH,PHARMA
37,IEX,219.00,-0.06,-4.89,56.27,48.63,107009.0,-9784.0,190170.0,-36.71,46.0,H-SC,13.54,136.0,-0.09,1.38,6.09,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.37,-19.27,92.59,55.47,181843.0,-46890.0,196396.0,-28.46,22.0,X-MC,13.96,64.0,-0.26,1.42,0.63,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.40,-10.85,43.71,28.11,78481.0,-21860.0,179550.0,-35.36,25.0,X-MC,13.46,55.0,-0.28,1.30,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.14,-24.46,103.18,53.50,235207.0,-73793.0,227958.0,-63.37,27.0,X-MC,3.79,58.0,-0.31,1.65,1.25,XY24,NTT,FMCG
75,TMPV,600.00,-1.63,-30.17,76.11,22.98,120059.0,-68154.0,157744.0,-27.96,27.0,X-LC,6.75,3.0,-0.57,1.14,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.34,-39.61,123.90,35.20,96270.0,-50970.0,77700.0,1.81,29.0,X-SC,16.46,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.44,-3.43,28.90,24.48,70288.0,-8646.0,243210.0,-81.54,39.0,X-SC,0.62,86.0,-0.12,1.76,18.91,X40N,NTT,MISC
12,BERGEPAINT,680.0,-1.02,-4.98,27.48,21.14,59364.0,-11312.0,216027.0,-11.64,36.0,X-MC,1.69,74.0,-0.19,1.56,21.76,XY24,NTT,PAINTS
20,DABUR,735.0,-0.74,-4.10,49.85,43.70,118589.0,-10176.0,237892.0,-11.99,32.0,X-MC,1.97,72.0,-0.09,1.72,10.33,XY24,BTT,FMCG
43,ITC,452.0,0.35,-1.67,12.66,10.78,29916.0,-4011.0,236307.0,-41.56,45.0,X-LC,2.05,5.0,-0.13,1.71,3.48,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.14,4.48,25.12,30.73,51706.0,8831.0,205836.0,-20.00,54.0,X-MC,2.31,75.0,0.17,1.49,17.82,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.27,-26.28,122.68,64.15,215193.0,-62541.0,175410.0,-56.75,30.0,X-SC,3.01,82.0,-0.29,1.27,0.0,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.34,-39.61,123.90,35.20,96270.0,-50970.0,77700.0,1.81,29.0,X-SC,16.46,92.0,-0.53,0.56,0.0,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-0.51,-21.15,79.46,41.51,165015.0,-55695.0,207670.0,-6.08,33.0,X-MC,7.53,61.0,-0.34,1.50,0.0,XY25,ATH,FMCG
53,PAGEIND,51605.08,-0.40,-10.85,43.71,28.11,78481.0,-21860.0,179550.0,-35.36,25.0,X-MC,13.46,55.0,-0.28,1.30,0.0,X40,ATH,APPARELS
75,TMPV,600.00,-1.63,-30.17,76.11,22.98,120059.0,-68154.0,157744.0,-27.96,27.0,X-LC,6.75,3.0,-0.57,1.14,0.0,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.68,-31.40,108.22,42.84,48261.0,-20411.0,44595.0,-54.82,37.0,X-SC,35.19,83.0,-0.42,0.32,0.89,XY24,NTT,MISC
59,RELAXO,1176.00,-0.80,-48.45,195.44,52.31,146254.0,-70327.0,74833.0,-45.12,38.0,X-SC,7.27,91.0,-0.48,0.54,0.17,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.34,-39.61,123.90,35.20,96270.0,-50970.0,77700.0,1.81,29.0,X-SC,16.46,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.28,-5.81,31.73,24.08,37942.0,-7371.0,119579.0,-13.35,30.0,X-SC,5.41,89.0,-0.19,0.87,12.67,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.56,-17.36,73.42,43.32,98823.0,-28272.0,134600.0,-29.05,34.0,X-SC,5.02,90.0,-0.29,0.97,3.35,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.73,-10.88,35.44,20.71,119466.0,-41142.0,337095.0,-23.30,64.0,X-LC,4.07,1.0,-0.34,2.44,12.64,X40,ATH,IT
41,INFY,1972.00,1.19,10.36,21.65,34.24,74756.0,32406.0,345294.0,-14.21,65.0,X-LC,4.56,2.0,0.43,2.50,19.50,X40,NTT,IT
75,TMPV,600.00,-1.63,-30.17,76.11,22.98,120059.0,-68154.0,157744.0,-27.96,27.0,X-LC,6.75,3.0,-0.57,1.14,0.00,XY24,NTT,AUTO
81,VBL,671.64,-0.73,-5.59,43.70,35.67,130314.0,-17641.0,298201.0,-16.86,47.0,X-LC,3.87,4.0,-0.14,2.16,7.50,X40N,ATH,FMCG
43,ITC,452.00,0.35,-1.67,12.66,10.78,29916.0,-4011.0,236307.0,-41.56,45.0,X-LC,2.05,5.0,-0.13,1.71,3.48,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.55,-38.10,114.69,32.89,54809.0,-29418.0,47789.0,-690.25,51.0,L-MC,6.25,259.0,-0.54,0.35,31.61,XR,NTT,BANKS
6,ASIANTILES,137.00,3.60,-11.54,105.37,81.67,87443.0,-10823.0,82987.0,7461.11,60.0,L-SC,12.43,271.0,-0.12,0.60,60.71,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.60,-4.12,27.29,22.05,25638.0,-4035.0,93945.0,-17.34,49.0,H-SC,6.78,164.0,-0.16,0.68,36.56,XR,ATH,FINANCE
13,BSOFT,831.70,0.41,-21.39,91.81,50.79,101512.0,-30085.0,110568.0,0.89,65.0,H-SC,6.60,151.0,-0.30,0.80,28.74,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.39,-16.07,19.17,0.02,28085.0,-28053.0,146506.0,123.21,42.0,M-SC,13.67,234.0,-1.00,1.06,27.43,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.41,-21.39,91.81,50.79,101512.0,-30085.0,110568.0,0.89,65.0,H-SC,6.60,151.0,-0.30,0.80,28.74,XR,ATH,IT
84,WIPRO,420.00,0.92,8.10,59.36,72.27,108728.0,13719.0,183167.0,-7.44,69.0,M-LC,2.80,99.0,0.13,1.33,14.77,XR,NTT,IT
41,INFY,1972.00,1.19,10.36,21.65,34.24,74756.0,32406.0,345294.0,-14.21,65.0,X-LC,4.56,2.0,0.43,2.50,19.50,X40,NTT,IT
61,ROUTE,2227.21,7.18,-44.64,205.37,69.05,151284.0,-59402.0,73664.0,-55.25,59.0,H-SC,22.33,140.0,-0.39,0.53,13.61,SR,ATH,IT
73,TCS,4389.97,0.73,-10.88,35.44,20.71,119466.0,-41142.0,337095.0,-23.30,64.0,X-LC,4.07,1.0,-0.34,2.44,12.64,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.37
1,25,44.85
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.81
MC    30.91
LC    24.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.61
X40      23.16
X40N     12.97
XR        9.81
XY25      9.11
AR        9.00
OX40N     7.54
SR        1.02
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.43
H-SC    23.78
X-LC    20.79
M-SC    11.59
X-SC     8.01
H-MC     4.80
L-SC     1.43
M-LC     1.33
M-MC     1.33
H-LC     1.12
L-LC     1.03
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.33,-8.88,43.89
IT,13.21,-15.19,73.11
FINANCE,9.44,-22.07,75.16
MISC,7.29,-29.06,81.85
ELECTRICAL,5.99,-13.17,54.63
PAINTS,5.82,-14.67,31.70
INSURANCE,4.80,-2.30,37.31
PHARMA,4.15,-2.35,34.57
AUTO,2.79,-35.69,82.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3243502.0,21
AR,1326569.0,10
XR,1306685.0,13
X40,1021783.0,14
X40N,892805.0,9
OX40N,749167.0,10
XY25,380370.0,6
SR,279405.0,2
MH,78007.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3692226.0,25
M-SC,1454236.0,15
X-MC,1449252.0,16
X-LC,852803.0,11
X-SC,810880.0,8
H-MC,414890.0,3
L-SC,263672.0,3
M-LC,108728.0,1
H-LC,78676.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1297938.0      6
           AR         902270.0      5
M-SC       XY24       832400.0      6
H-SC       XR         789282.0      7
X-MC       X40        500316.0      7
           XY24       413160.0      3
           X40N       351464.0      4
H-SC       OX40N      345324.0      4
X-LC       X40        327348.0      5
M-SC       OX40N      316643.0      5
X-SC       X40N       315365.0      3
           XY24       301396.0      3
H-SC       SR         279405.0      2
X-LC       X40N       225976.0      2
H-MC       AR         217824.0      2
X-LC       XY24       201542.0      2
H-MC       XY24       197066.0      1
X-SC       X40        194119.0      2
X-MC       XY25       184312.0      2
M-SC       XR         177394.0      2
L-SC       XR         176472.0      2
M-SC       AR         127799.0      2
M-LC       XR         108728.0      1
X-LC       XY25        97937.0      2
L-SC       OX40N       87200.0      1
H-LC       AR          78676.0      1
H-SC       MH          78007.0      1
M-MC       XY25        61208.0      1
L-MC       XR          54809.0      1
L-LC       XY25        36913.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
